In [21]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')



In [22]:
m = ['11', '12', '01', '02', '03']
station = 'Haukeliseter'

### Retrieval data

In [23]:
txt_dir = '../../Data/Retrieval/'
txt_filename = 'pulsed_minute_surface_acc_Haukeli.dat.txt'
txt_filename = 'upslope_minute_surface_acc_Haukeli.dat.txt'

In [24]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [25]:
Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)

In [26]:
obs_f = dict()
obs_wind_dir_10m = dict()
obs_wind_speed_10m = dict()
obs_wind_dir = dict()
obs_wind_speed = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_westerly_idx = dict()
obs_easterly_idx = dict()
obs_westerly = dict()
obs_easterly = dict()

for i in range(int(np.round((Haukeli.shape[0]-1)/int(1440/6+1)))):
    Date = int(Haukeli['Unnamed: 0'][i*int(1440/6+1)])
    precip_24 = (Haukeli['Unnamed: 1'][i*int(1440/6+1)])
    precip_24 = np.array([precip_24])
    
    column0 = Haukeli['Unnamed: 0'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column1 = Haukeli['Unnamed: 1'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column2 = Haukeli['Unnamed: 2'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column3 = Haukeli['Unnamed: 3'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column4 = Haukeli['Unnamed: 4'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column5 = Haukeli['Unnamed: 5'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]

    column0 = column0.convert_objects(convert_numeric=True)
    column1 = column1.convert_objects(convert_numeric=True)
    column2 = column2.convert_objects(convert_numeric=True)
    column3 = column3.convert_objects(convert_numeric=True)
    column4 = column4.convert_objects(convert_numeric=True)
    column5 = column5.convert_objects(convert_numeric=True)

    ret_diff = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
    ret_diff = np.array(ret_diff)

    ret_diff = ret_diff.flatten()/ret_diff.flatten().shape[0]
    

    

    obs_dirnc = '../../Data/Weathermast/Met_%s_%s.nc' %(Date,station)
    obs_f[Date] = netCDF4.Dataset(obs_dirnc, 'r')

    obs_wind_dir_10m[Date]   = (obs_f[Date].variables['10m_wind_dir_mast1'][:])
    obs_wind_speed_10m[Date] = (obs_f[Date].variables['10m_wind_speed_mast1'][:])

    obs_f[Date].close()

    filename = txt_filename.split('_')[0]+'_precip_%s.nc' %(Date)

    ### write netCDF file
    fw = netCDF4.Dataset(txt_dir+filename, 'w')
    ### create dimensions
    fw.createDimension('time', ret_diff.shape[0])
    fw.createDimension('24h_acc_sum', precip_24.shape[0])
    
    precip_w       = get_netCDF_variable(fw, 'precipitation', ret_diff, ('time',))
    total_precip_w = get_netCDF_variable(fw, '24h_precip_acc', precip_24, ('24h_acc_sum',) )
    
    fw.close()

In [27]:
Date_str = str(Date)

year = Date_str[0:4]
month = Date_str[4:6]
day = Date_str[6:]

In [28]:
        ## Create a rolling mean to smoothen out the data
        # attach the last 9 minutes from the previous day to calculate the rolling mean every 10 minutes
        if month != '11' and day == '01':
            if month == '12':
                previous_day = '30'
            elif month == '01' or month == '02':
                previous_day = '31'
            elif month == '03':
                previous_day = '28'
            if month == '01':
                previous_month = '12'
                previous_year = str(int(year)-1)
            elif month == '12':
                previous_month = str(int(month)-1)
                previous_year = year
            else:
                previous_month = '0%s' %(int(month)-1)
                previous_year = year
            
        elif int(day)-1 < 10:
            previous_day = '0%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        else:
            previous_day = '%s' %(int(day)-1)
            previous_month = month
            previous_year = year

In [29]:
        wd = (np.concatenate((obs_wind_dir_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_dir_10m[Date][:])))
        obs_wind_dir[Date] = (wd)
        ws = (np.concatenate((obs_wind_speed_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_speed_10m[Date][:])))
        obs_wind_speed[Date] = (ws)
        # Calculate the moving average. That is, take
        # the first ten values, average them, 
        # then drop the first and add the eleventh, etc.
        obs_wind_dir[Date] = pd.DataFrame(np.array(obs_wind_dir[Date])[:])
        obs_wind_dir_rolling[Date] = obs_wind_dir[Date].rolling(window=10,min_periods=7,axis=0).mean()
        obs_wind_dir_rolling[Date] = obs_wind_dir_rolling[Date].values
        obs_wind_dir_rolling[Date] = np.array(obs_wind_dir_rolling[Date])[9:,:]

        obs_wind_speed[Date] = pd.DataFrame(np.array(obs_wind_speed[Date])[:])
        obs_wind_speed_rolling[Date] = obs_wind_speed[Date].rolling(window=10,min_periods=7,axis=0).mean()
        obs_wind_speed_rolling[Date] = obs_wind_speed_rolling[Date].values
        obs_wind_speed_rolling[Date] = np.array(obs_wind_speed_rolling[Date])[9:,:]

In [30]:
        # Find wind regimes and associated precipitation pattern
        IDX = np.arange(obs_wind_dir_rolling[Date].shape[0])

        ## find all obs_westerly regimes (202.5 - 22.5 deg) 
        obs_westerly_idx[Date]  = IDX[np.logical_or(np.logical_and((obs_wind_dir_rolling[Date][:,0] >= 202.5), 
                                                                   (obs_wind_dir_rolling[Date][:,0] <= 360)),
                                                   (obs_wind_dir_rolling[Date][:,0] <22.5))]
        ## find all obs_easterly regimes (22.5 - 202.5 deg)
        obs_easterly_idx[Date]  = IDX[np.logical_and((obs_wind_dir_rolling[Date][:,0] >= 22.5),
                                                     (obs_wind_dir_rolling[Date][:,0] < 202.5))]

        ## find indices for all obs_westerly regimes 
        sub_idx = []

        for i in range(0,obs_westerly_idx[Date] .shape[0]-1):
            if obs_westerly_idx[Date] [i]+1 == obs_westerly_idx[Date] [i+1]:
                continue
            else:
                #print(i)
                sub_idx.append(i)
    
       
        ## find westerlies which are < 29 min and assign them to obs_easterly wind regime
        if len(obs_westerly_idx[Date]) == 0 and len(sub_idx) == 0:     # westerlies do not exist
            obs_westerly[Date] = []
            obs_easterly[Date] = obs_easterly_idx[Date]
        elif len(obs_westerly_idx[Date]) != 0 and len(sub_idx) == 0:   # westerlies exist only for one time period a day
            obs_westerly[Date] = []
            obs_easterly[Date] = obs_easterly_idx[Date]
            
            obs_westerly_duration = obs_westerly_idx[Date][-1] - obs_westerly_idx[Date][0]
            if obs_westerly_duration >= 28:
                obs_westerly[Date] = np.concatenate((obs_westerly[Date],obs_westerly_idx[Date][:]))
            elif obs_westerly_duration < 28:
                obs_easterly[Date] = np.concatenate((obs_easterly[Date],obs_westerly_idx[Date][:]))
        else:
            obs_westerly[Date] = []
            obs_easterly[Date] = obs_easterly_idx[Date]

            for i in np.arange(0,np.array(sub_idx).shape[0]+1):                # westerlies and easterlies interchange daily
                if i == 0:
                    obs_westerly_duration = (obs_westerly_idx[Date] [ sub_idx[0]] - \
                                         obs_westerly_idx[Date] [0])
                elif i == np.array(sub_idx).shape[0]:
                    obs_westerly_duration = (obs_westerly_idx[Date] [-1] - \
                                         obs_westerly_idx[Date] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    obs_westerly_duration = (obs_westerly_idx[Date] [ sub_idx[i]] - \
                                         obs_westerly_idx[Date] [ sub_idx[i-1]+1])
                if obs_westerly_duration >= 28:
                    if i == 0:
                        obs_westerly[Date] = np.concatenate((obs_westerly[Date], 
                                                             obs_westerly_idx[Date] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_westerly[Date] = np.concatenate((obs_westerly[Date], 
                                                             obs_westerly_idx[Date] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                        obs_westerly[Date] = np.concatenate((obs_westerly[Date], 
                                                             obs_westerly_idx[Date] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
                elif obs_westerly_duration < 28:
                    if i == 0:
                        obs_easterly[Date] = np.concatenate((obs_easterly[Date], 
                                                             obs_westerly_idx[Date] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_easterly[Date] = np.concatenate((obs_easterly[Date],
                                                             obs_westerly_idx[Date] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                   # print(obs_westerly_duration)
                        obs_easterly[Date] = np.concatenate((obs_easterly[Date],
                                                             obs_westerly_idx[Date] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
        
        if len(obs_westerly[Date]) == 0:    # only easterlies
            obs_easterly[Date]  = obs_easterly[Date].astype(int)
        elif len(obs_easterly[Date]) == 0:  # only westerlies
            obs_westerly[Date]  = obs_westerly[Date].astype(int)
        else:    # both easterlies and westerlies
            obs_easterly[Date]  = obs_easterly[Date].astype(int)
            obs_westerly[Date]  = obs_westerly[Date].astype(int)
            
        # assign obs_diff values to wind regime

In [44]:
np.concatenate([obs_easterly[Date],obs_westerly[Date]])

array([ 239,  240,  241, ..., 1393, 1394, 1395])

In [45]:
obs_westerly[Date]

array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
         44,   45,   46,   47,   48,   49,   50,   51,   52,   53,   54,
         55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
         66,   67,   68,   69,   70,   71,   72,   73,   74,   75,   76,
         77,   78,   79,   80,   81,   82,   83,   84,   85,   86,   87,
         88,   89,   90,   91,   92,   93,   94,   95,   96,   97,   98,
         99,  100,  101,  102,  103,  104,  105,  106,  107,  108,  109,
        110,  111,  112,  113,  114,  115,  116,  117,  118,  119,  120,
        121,  122,  123,  124,  125,  126,  127,  128,  129,  130,  131,
        132,  133,  134,  135,  136,  137,  138,  139,  140,  141,  142,
        143,  144,  145,  146,  147,  148,  149,  1

In [41]:
ret_diff[obs_easterly[Date],]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [39]:
ret_diff[obs_westerly[Date],].shape

(493,)

In [ ]:
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
            obs_total_precip[int(year+month+day)] = np.nan
            obs_precip_east[int(year+month+day)] =  np.nan
            obs_precip_west[int(year+month+day)] =  np.nan
        else:
            obs_total_precip[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
                                                np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:])
            obs_precip_east[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:])
            obs_precip_west[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:])
            